# Practica 2: Sistemas Multimedia
## Jose Juan Bastida Perez


**Importamos las librerias necesarias**

Pasos previos:
* Clonamos el repositorio con el comando git clone
* Creamos el entorno de conda con el comando conda create --name= seguido del nombre del archivo que vamos a usar para definir el entorno y lo activamos
* Una vez esta activado podemos lanzar jupyter-lab y creamos nuestro notebook

In [ ]:
from scipy.io import wavfile
import IPython
import os
import numpy as np

**Ahora tenemos que definir donde guardaremos los audios o de donde los cogeremos**


In [ ]:
cwd = os.getcwd()  #guardamos en cwd la ruta actual
audio_input_path = os.path.join(cwd, os.path.join('audio', 'ejemplos'))  
audio_output_path = os.path.join(cwd, os.path.join('audio', 'output'))

print(f'Directorio con los audios de entrada: {audio_input_path}')
print(f'Directorio donde guardaremos los audios generados: {audio_output_path}\n')

**Ahora vamos a cargar un archivo de audio de la carpeta ejemplos**


In [ ]:

filename = os.path.join(audio_input_path,'breaking_bad.wav')
sample_rate, audio_data = wavfile.read(filename)  #en sample rate se guardara la frecuencia y en audio data los datos de la cancion
print(f'Frecuencia de muestreo (sample rate): {sample_rate/1000} kHz')

**Mostramos la frecuencia de muestreo y en audio data tenemos la información de la cancion**





 


**Para reproducirlo usamos la funcion display.Audio donde es necesario pasarle el audio data y la frecuencia de muestreo**

In [ ]:
IPython.display.Audio(audio_data.T, rate=sample_rate) # .T se pasa únicamente si es audio estéreo.

**A continuacion vamos a mostrar la informacion principal de la onda, la cual la tenemos en audio_data**


In [ ]:
print('Datos de audio (estereo):')
print(f'- Tamaño:     {audio_data.shape}') #aqui se enseña el numero de muestras y el numero de canales
print(f'- 1º canal:   {audio_data[:10, 0]}...') #aqui mostramos 5 elementos del primer canal
print(f'- 2º canal:   {audio_data[:10, 1]}...') #aqui mostramos 5 elementos del primer canal
print(f'- Resolucion: {type(audio_data[0,0])}\n') #mostramos el tipo de dato

**Para pasar un audio a mono, lo que tendriamos que hacer seria juntar los sonidos en un mismo canal, por lo tanto calculamos la media de ambos canales**


In [ ]:

# Convertimos a mono mediante la media por canal (simplificacion).
new_data_mono = audio_data.mean(axis=1)  # Se hace la media de dos muestras, de ambos canales
print('Nuevos datos de audio (mono):')
print(f'- Nuevo tamaño: {new_data_mono.shape}')  #aqui se enseña el numero de muestras y el numero de canales (en este caso solo hay 1)
print(f'- Canal unico:  {new_data_mono[:5]}...') #muestra 5 elementos del canal unico

# Mantenemos la misma resolucion que antes y la mostramos para comprobar que este correcta.
new_data_mono = new_data_mono.astype(np.int16)
print(f'- Resolucion:   {type(new_data_mono[0])}\n')

**Ahora procedemos a guardar el archivo en la ruta que nos indica audio_output_path**

In [ ]:
# Guardamos el archivo mono a un fichero de tipo wav.
wavfile.write(
    filename=os.path.join(audio_output_path, 'ejemplo_mono.wav'),
    rate=sample_rate,
    data=new_data_mono
)

**Volvemos a escucharlo**

In [ ]:
IPython.display.Audio(new_data_mono, rate=sample_rate)

**Grafica**

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Cargar datos de audio estéreo y mono (asumiendo que ya has cargado los datos)
# audio_data_stereo y audio_data_mono son los arrays NumPy que contienen los datos de audio

# Configuración del tamaño de la figura
plt.figure(figsize=(12, 6))

# Graficar audio estéreo
plt.subplot(2, 1, 1)  # Configurar subplot para el audio estéreo
plt.plot(np.arange(len(audio_data)) / sample_rate, audio_data[:, 0], label='Canal izquierdo')
plt.plot(np.arange(len(audio_data)) / sample_rate, audio_data[:, 1], label='Canal derecho')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Audio Estéreo')
plt.legend()

# Graficar audio mono
plt.subplot(2, 1, 2)  # Configurar subplot para el audio mono
plt.plot(np.arange(len(new_data_mono)) / sample_rate, new_data_mono, color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Amplitud')
plt.title('Audio Mono')

# Ajustar diseño y mostrar la gráfica
plt.tight_layout()
plt.show()


**La principal diferencia entre el sonido en estereo se reparte entre los dos canales de sonido, mientras que el sonido en mono solo va por un canal.**